In [ ]:
# Import required libraries
import os as os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set environment
np.random.seed(1234)
pd.set_option('display.max_columns', None)
os.chdir('E:/backup17092018/Myappdir/Myprojects/Data Science/Python Employee Attrition')
os.getcwd()

In [ ]:
# Read the .csv file
file_name = ('HR-Employee-Attrition.csv')
df = pd.read_csv(file_name)

Data Wrangling

In [ ]:
df.head()

In [ ]:
# Glimpse of the dataset
df.dtypes

In [ ]:
df.shape

In [ ]:
df.dtypes.value_counts()

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [ ]:
df.info()

In [ ]:
df.describe(include = 'all')

In [ ]:
df2 = df

In [ ]:
# Separate categorical and numerical columns
cat_cols = df2.select_dtypes(include=['object']).columns
num_cols = df2.select_dtypes(include=['int64']).columns
print("String fields:")
print(cat_cols)
print("Numerical fields:")
print(num_cols)

In [ ]:
# Column names: remove starting and ending white spaces and convert to lower case
# df.columns = df.columns.str.strip().str.lower().str.replace(' ,-', '_')
df2.columns = df2.columns.str.replace(' ', '_')

In [ ]:
df2.columns

In [ ]:
# Remove empty rows/columns
df2.dropna(how="all", axis=0, inplace=True) # Rows
df2.dropna(how="all", axis=1, inplace=True) # Columns
df2.shape

In [ ]:
# Remove duplicate rows
df2.drop_duplicates(keep='first', inplace=True)
df2.shape

In [ ]:
# Trim extra spaces from categorical columns
df2[cat_cols] = df2[cat_cols].apply(lambda x: x.str.strip())

In [ ]:
# Count unique values
df2.nunique().sort_values()

In [ ]:
# Show a sample of unique values for each column
print(df2.apply(lambda col: col.unique()))

In [ ]:
# Check for nulls
percent_missing = df2.isnull().sum() * 100 / len(df)
missing_value_df2 = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df2

In [ ]:
# Replace numerical nulls with mean
df2 = df2.apply(lambda num_col: num_col.replace(np.nan, np.mean(num_col)) if np.issubdtype(num_col.dtype, np.number) else num_col)

In [ ]:
# Replace categorical nulls with the most used value
df2 = df2.apply(lambda str_col: str_col.replace(np.nan, str_col.fillna(str_col.value_counts().index[0])) if np.issubdtype(str_col.dtype, np.string_) else str_col)

In [ ]:
# Replace 'Attrition' Yes=1 No=0
df2['Attrition'] = df2['Attrition'].apply(lambda x: 0 if x == 'No' else 1)

In [ ]:
# Convert 'Attrition' to numeric
df2['Attrition'] = df2['Attrition'].astype('int64')

In [ ]:
# Replace 'OverTime' Yes=1 No=0
df2['OverTime'] = df2['OverTime'].apply(lambda x: 0 if x == 'N' else 1)

In [ ]:
# Convert 'OverTime' to numeric
df2['OverTime'] = df2['OverTime'].astype('int64')

In [ ]:
# Remove unwanted columns
df2 = df2.drop(labels = ['Over18', 'StandardHours', 'EmployeeCount'], axis=1)

In [ ]:
# Moving Response Variable to the last column
front = df2['Attrition']
df2.drop(labels=['Attrition'], axis=1,inplace=True)
df2.insert(len(df2.columns),'Attrition',front)
df2.head()

In [ ]:
# Changing object types to categories
cols = ['BusinessTravel','Department','EducationField','Gender','JobRole','MaritalStatus','OverTime']
for col in cols:
    df2[col] = df2[col].astype('category')

In [ ]:
df2.info()

In [ ]:
df2[cols].nunique()

In [ ]:
for col in cols:
    display(df2[col].value_counts())

In [ ]:
# This function creates a dataframe for each feature in the dataset
def table_df(x): 
    feature = df2.groupby(x).agg({'Attrition':['size', 'sum', 'mean']})
    feature.columns = [' '.join(col).strip() for col in feature.columns.values]
    feature = feature.reset_index()
    feature['Attrition mean'] = feature['Attrition mean']*100
    feature.columns = [x, 'Total Employees ', 'Attrition Number', 
                       '% of Attrition in the Relevant Group'] 
    feature['% of Attrition in the Company'] = (feature['Attrition Number'] / 237)*100
    return feature

In [ ]:
# This function creates graph for each feature in the dataset 

def graph_df(x, y = 'bar', z = 0):
    feature = df2.groupby(x).agg({'Attrition':['size', 'sum', 'mean']})
    feature.columns = [' '.join(col).strip() for col in feature.columns.values]
    feature = feature.reset_index()
    feature['Attrition mean'] = feature['Attrition mean']*100
    feature.columns = [x, 'Total Employee', 'Attrition Number', 
                       '% of Attrition in the Relevant Group'] 
    
    fig0, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    feature.set_index(feature[x], drop=True, inplace=True)
    feature['Total Employee'].plot(kind = y, stacked = True, ax = ax1, 
                                   colormap = 'winter', rot = z)
    feature['% of Attrition in the Relevant Group'].plot(use_index = False, kind = 'line', ax = ax2, colormap='Spectral')
    plt.title('Attrition compared to' + " " + x  + " " + "Groups")
    ax1.set_xlabel(x)
    ax1.yaxis.label.set_color('blue')
    ax1.set_ylabel("Total Employees")
    ax2.set_ylabel("Attrition Percentage in the Relevant Group")
    ax2.yaxis.label.set_color('red')
    plt.tight_layout()
    plt.show()

In [ ]:
# Create 'Age_Bin' bins
table_df(x = 'Age')
bins = [17, 21, 25, 35, 60]
bin_names = ['(18-21)', '(22-25)', '(26-35)', '(36-60)']
df2['Age_Bin'] = pd.cut(df2['Age'], bins, labels = bin_names)

In [ ]:
# Create 'DistanceFromHome_Bin' bins
table_df(x = 'DistanceFromHome')
bins = [0, 3, 6, 10, 29]
bin_names = ['1-3', '4-6', '7-10', '10+']
df2['DistanceFromHome_Bin'] = pd.cut(df2['DistanceFromHome'], bins, labels = bin_names)

In [ ]:
# Create 'Education_Bin' bins
table_df(x = 'Education')
df2['Education_Bin'] = df2['Education'].map({1:'Below College', 2:'College', 3:'Bachelor', 4:'Master', 5:'Doctor'})

In [ ]:
# Create 'EnvironmentSatisfaction_Bin' bins
table_df(x = 'EnvironmentSatisfaction')
df2['EnvironmentSatisfaction_Bin'] = df2['EnvironmentSatisfaction'].map({1:'Low', 2:'Medium', 3:'High', 4:'Very High'})

In [ ]:
# Create 'JobInvolvement_Bin' bins
table_df(x = 'JobInvolvement')
df2['JobInvolvement_Bin'] = df2['JobInvolvement'].map({1:'Low', 2:'Medium', 3:'High', 4:'Very High'})

In [ ]:
# Create 'JobLevel_Bin' bins
table_df(x = 'JobLevel')
df2['JobLevel_Bin'] = df2['JobLevel'].map({1:'Level-1', 2:'Level-2', 3:'Level-3', 4:'Level-4', 5:'Level-5'})

In [ ]:
# Create 'JobSatisfaction_Bin' bins 
table_df(x = 'JobSatisfaction')
df2['JobSatisfaction_Bin'] = df2['JobSatisfaction'].map({1:'Low', 2:'Medium', 3:'High', 4:'Very High'})

In [ ]:
# Create 'NumCompaniesWorked_Bin' bins
table_df(x = 'NumCompaniesWorked')
bins = [0, 1, 2, 3, 4, 5, 10]
group_names = ['0', '1', '2', '3', '4', '5+']             
df2['NumCompaniesWorked_Bin'] = pd.cut(df2['NumCompaniesWorked'], bins, labels = group_names, include_lowest = True, right = False)

In [ ]:
# Create 'MonthlyIncome_Bin' bins
table_df(x = "MonthlyIncome")
bins = [1000, 2000, 3000, 4000, 5000, 7500, 9000, 11000, 15000, 20000]
group_names = ['1000-2000', '2000-3000', '3000-4000', '4000-5000', '5000-7500', 
               '7500-9000', '9000-11000', '11000-15000', '15000-20000']             
df2["MonthlyIncome_Bin"] = pd.cut(df2["MonthlyIncome"], bins, labels = group_names)

In [ ]:
# Create 'PercentSalaryHike_Bin' bins
table_df(x = 'PercentSalaryHike')
bins = [10, 14, 18, 22, 25]
group_names = ['11-14', '15-18', '19-22', '23-25']             
df2['PercentSalaryHike_Bin'] = pd.cut(df2['PercentSalaryHike'], bins, labels = group_names)

In [ ]:
# Labeling Relationship Satisfaction Levels
table_df(x = 'RelationshipSatisfaction')
df2['RelationshipSatisfaction_Bin'] = df2['RelationshipSatisfaction'].map({1:'Low', 2:'Medium', 3:'High', 4:'Very High'})

In [ ]:
# Total Working Years & Attrition Table
table_df(x = 'TotalWorkingYears')
bins = [0, 1, 5, 10, 15, 20, 40]
group_names = ['1', '2-5', '6-10', '11-15', '16-20', '20+']             
df2['TotalWorkingYears_Bin'] = pd.cut(df2['TotalWorkingYears'], bins, labels = group_names, include_lowest = True)

In [ ]:
# Labeling Work Life Balance Groups
table_df(x = 'WorkLifeBalance')
df2['WorkLifeBalance_Bin'] = df2['WorkLifeBalance'].map({1:'Bad', 2:'Good', 3:'Better', 4:'Best'})

In [ ]:
# Labeling Years at Company Groups
table_df(x = 'YearsAtCompany')
bins = [0, 1, 5, 10, 40]
group_names = ['1', '2-5', '6-10', '10+']             
df2['YearsAtCompany_Bin'] = pd.cut(df2['YearsAtCompany'], bins, labels = group_names, include_lowest = True)

In [ ]:
# Labeling Years in Current Role Groups
table_df(x = 'YearsInCurrentRole')
bins = [0, 1, 4, 6, 7, 10, 18]
group_names = ['1', '2-4', '5-6', '7', '8-10', '10+']             
df2['YearsInCurrentRole_Bin'] = pd.cut(df2['YearsInCurrentRole'], bins, labels = group_names, include_lowest = True)

In [ ]:
# Labeling Years in Current Role Groups
table_df(x = 'YearsSinceLastPromotion')
bins = [0, 1, 2, 3, 4, 7, 8, 16]
group_names = ['0', '1', '2', '3', '4-6', '7', '8+']             
df2['YearsSinceLastPromotion_Bin'] = pd.cut(df2['YearsSinceLastPromotion'], bins, labels = group_names, include_lowest = True, right = False)

In [ ]:
# Labeling Years with Current Manager Groups
table_df(x = 'YearsWithCurrManager')
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 18]
group_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10+']             
df2['YearsWithCurrManager_Bin'] = pd.cut(df2['YearsWithCurrManager'], bins, labels = group_names, include_lowest = True, right = False)

Exploratory Data Analysis

In [ ]:
# Target Variable
sns.countplot(x=df2['Attrition'])
# target = df['Attrition'].apply(lambda x: 'Active' if x == 0 else 'Churned')
# sns.countplot(target)
plt.title('Attrition in the Company')
plt.xlabel('Attrition Status')
plt.ylabel('Employee Number')
plt.show()
df['Attrition'].value_counts()

In [ ]:
table_df(x = 'Age_Bin')

In [ ]:
graph_df(x = 'Age_Bin')

In [ ]:
# Business Travel & Attrition
table_df(x = 'BusinessTravel')

In [ ]:
graph_df(x = 'BusinessTravel')

In [ ]:
# Gender & Attrition
table_df(x = 'Gender')

In [ ]:
graph_df(x = 'Gender')

In [ ]:
table_df(x = 'YearsWithCurrManager_Bin')

In [ ]:
graph_df(x = 'YearsWithCurrManager_Bin')

In [ ]:
df2.head()

In [ ]:
df2.columns.tolist()

In [ ]:
# Save cleaned file
file_name = ('HR-Employee-Attrition-Cleaned.csv')
df2.to_csv(file_name, header=True)